In [1]:
import cv2
import numpy as np
import pyaudio
import wave
import threading
import keyboard

In [2]:
# Create a VideoCapture object to access the camera 
frame_index = 1
cap = cv2.VideoCapture(0) #0 means webcam 

while True:
    ret, frame = cap.read()
    cv2.imshow("This is sepehr webcam", frame)

    #Press the "c" key to capture a frame
    if cv2.waitKey(1) & 0xFF == ord("c"):  
        cv2.imwrite(f"captured_frame.jpg", frame)
        break
cap.release()
cv2.destroyAllWindows()

In [3]:
# Set the audio recording parameters
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 30
class AudioRecorder:
    def __init__(self):
        self.frames = []
        self.recording = False

    def start_recording(self):
        self.recording = True
        self.frames = []
        threading.Thread(target=self._record_audio).start()

    def stop_recording(self):
        self.recording = False

    def _record_audio(self):
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)
        while self.recording:
            data = stream.read(CHUNK)
            self.frames.append(data)
        stream.stop_stream()
        stream.close()
        p.terminate()

    def save_recording(self, filename):
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        output=True)
        wf = wave.open(filename, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(self.frames))
        wf.close()
        stream.stop_stream()
        stream.close()
        p.terminate()

In [4]:
# Create an instance of the AudioRecorder class
audio_recorder = AudioRecorder()
keyboard.on_press_key('r', lambda _: audio_recorder.start_recording())

# Stop recording and save the audio file
keyboard.on_press_key('s', lambda _: audio_recorder.stop_recording())

# Wait for the recording to complete
keyboard.wait('esc')

# Save the recorded audio as a file
audio_recorder.save_recording("recorded_audio.wav")